<a href="https://colab.research.google.com/github/MuhammadShaheer-khan/check/blob/master/check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# To determine which version you're using:
!pip show tensorflow

# For the current version: 
!pip install --upgrade tensorflow

# For a specific version:
!pip install tensorflow==1.2

# For the latest nightly build:
!pip install tf-nightly

Name: tensorflow
Version: 2.2.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: protobuf, six, numpy, h5py, opt-einsum, wrapt, scipy, tensorflow-estimator, grpcio, tensorboard, keras-preprocessing, absl-py, google-pasta, gast, termcolor, wheel, astunparse
Required-by: fancyimpute
Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.2.0)
     |████████████████████████████████| 35.0MB 72.9MB/s 
     |████████████████████████████████| 245kB 48.7MB/s 
     |████████████████████████████████| 890kB 35.6MB/s 
  Created wheel for markdown: filename=Markdown-2.2.0-cp36-none-any.whl size=136298 sha256=1844b79eeaa3e174b40fcd4ed69e6daaa4df18738ae913ec37e1b7b75dd49ddd
  Stored in directory: /root/.cache/pip/wheels/b6/52/17/f0af18e3e0ec6fa60b361ffed15b4c3468f6f3

In [2]:
pip install ta

  Created wheel for ta: filename=ta-0.5.25-cp36-none-any.whl size=24880 sha256=d8b1c5f531041ae5cf93ed74095ffc6886c34d4e594486c490fe9eb51c7a2dbd
  Stored in directory: /root/.cache/pip/wheels/2e/93/b7/cf649194508e53cee4145ffb949e9f26877a5a8dd12db9ed5b
Successfully built ta


In [3]:
# Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import timedelta
from sklearn.preprocessing import RobustScaler
plt.style.use("bmh")


import ta

# Neural Network library
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

Using TensorFlow backend.


In [4]:
from google.colab import files
uploaded = files.upload()
import io
df = pd.read_csv(io.BytesIO(uploaded['SPY.csv']))

Saving SPY.csv to SPY.csv


In [5]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-07-08,297.010010,298.260010,296.220001,296.820007,291.010712,45841800
1,2019-07-09,295.540009,297.519989,295.480011,297.190002,291.373474,41101300
2,2019-07-10,298.369995,299.660004,297.779999,298.609985,292.765656,58448500
3,2019-07-11,299.320007,299.579987,298.200012,299.309998,293.451996,50826100
4,2019-07-12,299.850006,300.730011,299.510010,300.649994,294.765747,40326000


In [17]:
# after conversion

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 251 entries, 2019-07-08 to 2020-07-02
Data columns (total 73 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Close                      251 non-null    float64
 1   volume_adi                 251 non-null    float64
 2   volume_obv                 251 non-null    float64
 3   volume_cmf                 251 non-null    float64
 4   volume_fi                  251 non-null    float64
 5   momentum_mfi               251 non-null    float64
 6   volume_em                  251 non-null    float64
 7   volume_sma_em              251 non-null    float64
 8   volume_vpt                 251 non-null    float64
 9   volume_nvi                 251 non-null    float64
 10  volume_vwap                251 non-null    float64
 11  volatility_atr             251 non-null    float64
 12  volatility_bbm             251 non-null    float64
 13  volatility_bbh             251 

In [18]:
df.tail()

,Close,volume_adi,volume_obv,volume_cmf,volume_fi,momentum_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_adx,trend_adx_pos,trend_adx_neg,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_cci,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,momentum_rsi,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,others_dr,others_dlr,others_cr
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-06-26,-0.009386,0.262485,0.161485,-0.412893,-2.776234,-1.244637,-0.759535,-2.915370,-1.473601,1.321035,0.443646,0.696818,0.591734,0.662552,0.345810,0.417876,-1.318727,0.0,0.0,0.405836,0.596289,0.205485,0.764479,-0.996681,0.0,1.0,0.422434,-1.428701,0.024148,0.653009,-1.736954,0.384259,0.528883,0.431430,0.439054,-0.562032,-0.553532,0.651590,-0.602056,1.081599,-0.898051,0.744276,0.128762,-1.214537,-2.177401,0.529255,1.061574,-1.696045,0.377603,0.669573,0.495458,0.325464,-0.656118,-4.472616,-0.533333,-0.285714,-0.12,0.017086,0.871993,0.0,0.0,-0.672988,-0.138705,-0.798883,-1.394698,-1.088480,-1.394698,-0.144562,0.700356,-1.926734,-2.097074,-2.159749,-0.009386
2020-06-29,0.197559,0.407038,0.300251,-0.313903,-1.810644,-1.022246,-0.727064,-3.073069,-1.332893,1.387319,0.402827,0.671718,0.589028,0.664331,0.339536,0.429690,-0.959202,0.0,0.0,0.399777,0.563344,0.223892,0.611446,-0.668697,0.0,0.0,0.422682,-1.083456,-0.082214,0.521309,-1.680391,0.398834,0.548369,0.408095,0.434941,-0.529577,-0.643088,0.521645,-0.599693,1.145022,-0.929010,0.600228,0.187828,-1.127004,-1.379635,0.376566,0.943147,-1.799694,0.341880,0.669573,0.474893,0.325464,-0.656118,-4.472616,-0.600000,-0.285714,-0.16,0.029039,0.871993,0.0,0.0,-0.437457,-0.185603,-0.531825,-1.030433,-1.070651,-1.030433,-0.340001,0.695359,0.001732,1.101626,1.112870,0.197559
2020-06-30,0.380572,0.498842,0.497501,-0.368401,-0.832170,-0.718926,2.413005,-2.316038,1.629837,1.387319,0.369877,0.664783,0.589723,0.664499,0.340542,0.428406,-0.651949,0.0,0.0,0.379096,0.534949,0.211466,0.571904,-0.286711,0.0,0.0,0.422618,-0.778138,-0.091403,0.414003,-1.390509,0.414545,0.574648,0.417028,0.447807,-0.580025,-0.296105,0.325422,-0.368840,0.874210,-0.658834,0.476669,0.215313,-0.721033,-0.762729,0.241035,0.815403,-1.818672,0.341880,0.732565,0.503154,0.325464,-0.656118,-4.472616,-0.666667,1.285714,-1.08,0.040753,0.871993,0.0,0.0,-0.244354,-0.192681,-0.203943,-0.381831,-0.918247,-0.381831,-0.533502,0.690014,0.024020,0.944556,0.955088,0.380572
2020-07-01,0.481933,0.502769,0.623434,-0.440365,-0.470215,-0.345844,1.078376,-0.173537,1.190982,1.419374,0.394061,0.594807,0.585601,0.659894,0.338077,0.425923,-0.474363,0.0,0.0,0.371012,0.523445,0.206970,0.554088,-0.073213,0.0,0.0,0.422996,-0.609038,-0.059761,0.334861,-1.055813,0.427682,0.597917,0.440469,0.468953,-0.649149,-0.225692,0.220698,0.301190,0.615624,-0.143383,0.378313,0.191651,-0.455994,0.545523,0.130407,0.683833,-1.746054,0.341880,0.768560,0.519303,0.325464,-0.547110,-4.208396,-0.733333,1.214286,-1.08,0.052232,0.871993,0.0,0.0,-0.141173,-0.181815,-0.122669,-0.145640,-0.493750,-0.145640,-0.530201,0.688926,-0.038492,0.461642,0.468134,0.481933
2020-07-02,0.562178,0.415700,0.744058,-0.571975,-0.223002,0.068670,1.526921,0.516853,0.484789,1.444752,0.428962,0.568728,0.587761,0.662586,0.339099,0.428003,-0.342395,0.0,0.0,0.379722,0.53

In [6]:
## Datetime conversion
df['Date'] = pd.to_datetime(df.Date)

# Setting the index
df.set_index('Date', inplace=True)

# Dropping any NaNs
df.dropna(inplace=True)



## Technical Indicators

# Adding all the indicators
df = ta.add_all_ta_features(df, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)

# Dropping everything else besides 'Close' and the Indicators
df.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)

# Only using the last 1000 days of data to get a more accurate representation of the current market climate
df = df.tail(1000)



## Scaling

# Scale fitting the close prices separately for inverse_transformations purposes later
close_scaler = RobustScaler()

close_scaler.fit(df[['Close']])

# Normalizing/Scaling the DF
scaler = RobustScaler()

df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)

/usr/local/lib/python3.6/dist-packages/ta/trend.py:608: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
/usr/local/lib/python3.6/dist-packages/ta/trend.py:612: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])


In [7]:
def split_sequence(seq, n_steps_in, n_steps_out):
    """
    Splits the multivariate time sequence
    """
    
    # Creating a list for both variables
    X, y = [], []
    
    for i in range(len(seq)):
        
        # Finding the end of the current sequence
        end = i + n_steps_in
        out_end = end + n_steps_out
        
        # Breaking out of the loop if we have exceeded the dataset's length
        if out_end > len(seq):
            break
        
        # Splitting the sequences into: x = past prices and indicators, y = prices ahead
        seq_x, seq_y = seq[i:end, :], seq[end:out_end, 0]
        
        X.append(seq_x)
        y.append(seq_y)
    
    return np.array(X), np.array(y)
  
  
def visualize_training_results(results):
    """
    Plots the loss and accuracy for the training and testing data
    """
    history = results.history
    plt.figure(figsize=(16,5))
    plt.plot(history['val_loss'])
    plt.plot(history['loss'])
    plt.legend(['val_loss', 'loss'])
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()
    
    plt.figure(figsize=(16,5))
    plt.plot(history['val_accuracy'])
    plt.plot(history['accuracy'])
    plt.legend(['val_accuracy', 'accuracy'])
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.show()
    
    
def layer_maker(n_layers, n_nodes, activation, drop=None, d_rate=.5):
    """
    Creates a specified number of hidden layers for an RNN
    Optional: Adds regularization option - the dropout layer to prevent potential overfitting (if necessary)
    """
    
    # Creating the specified number of hidden layers with the specified number of nodes
    for x in range(1,n_layers+1):
        model.add(LSTM(n_nodes, activation=activation, return_sequences=True))

        # Adds a Dropout layer after every Nth hidden layer (the 'drop' variable)
        try:
            if x % drop == 0:
                model.add(Dropout(d_rate))
        except:
            pass
          
          
def validater(n_per_in, n_per_out):
    """
    Runs a 'For' loop to iterate through the length of the DF and create predicted values for every stated interval
    Returns a DF containing the predicted values for the model with the corresponding index values based on a business day frequency
    """
    
    # Creating an empty DF to store the predictions
    predictions = pd.DataFrame(index=df.index, columns=[df.columns[0]])

    for i in range(n_per_in, len(df)-n_per_in, n_per_out):
        # Creating rolling intervals to predict off of
        x = df[-i - n_per_in:-i]

        # Predicting using rolling intervals
        yhat = model.predict(np.array(x).reshape(1, n_per_in, n_features))

        # Transforming values back to their normal prices
        yhat = close_scaler.inverse_transform(yhat)[0]

        # DF to store the values and append later, frequency uses business days
        pred_df = pd.DataFrame(yhat, 
                               index=pd.date_range(start=x.index[-1], 
                                                   periods=len(yhat), 
                                                   freq="B"),
                               columns=[x.columns[0]])

        # Updating the predictions DF
        predictions.update(pred_df)
        
    return predictions


def val_rmse(df1, df2):
    """
    Calculates the root mean square error between the two Dataframes
    """
    df = df1.copy()
    
    # Adding a new column with the closing prices from the second DF
    df['close2'] = df2.Close
    
    # Dropping the NaN values
    df.dropna(inplace=True)
    
    # Adding another column containing the difference between the two DFs' closing prices
    df['diff'] = df.Close - df.close2
    
    # Squaring the difference and getting the mean
    rms = (df[['diff']]**2).mean()
    
    # Returning the sqaure root of the root mean square
    return float(np.sqrt(rms))

In [8]:
# How many periods looking back to learn
n_per_in  = 90
# How many periods to predict
n_per_out = 30
# Features 
n_features = df.shape[1]
# Splitting the data into appropriate sequences
X, y = split_sequence(df.to_numpy(), n_per_in, n_per_out)

In [16]:
## Creating the NN

# Instatiating the model
model = Sequential()

# Activation
activ = "tanh"

# Input layer
model.add(LSTM(90, 
               activation=activ, 
               return_sequences=True, 
               input_shape=(n_per_in, n_features)))

# Hidden layers
layer_maker(n_layers=1, 
            n_nodes=30, 
            activation=activ)

# Final Hidden layer
model.add(LSTM(60, activation=activ))

# Output layer
model.add(Dense(n_per_out))

# Model summary
model.summary()

# Compiling the data with selected specifications
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

## Fitting and Training
res = model.fit(X, y, epochs=50, batch_size=128, validation_split=0.1)

AttributeError: ignored